# Progetto Computer Science

*Nome:* Christian

*Cognome:* Uccheddu

*Matricola:* 800428


**Setup dell'ambiente**

Importazione di tutte le librerie utili al funzionamento del programma e settaggio dei path in cui sono presenti i dati in modo da essere facilmente cambiabili all'inizio del programma.

In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
loans_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans.csv"
loans_lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans_lenders.csv"
lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/lenders.csv"
country_stats_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/country_stats.csv" 

**1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.**

Per questo compito mi occupo di importare il file loans_lenders, di analizzarne la struttura e successivamente decidere come procedere.

In [3]:
loans_lenders = pd.read_csv(loans_lenders_path)

In [4]:
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


Vedendo la struttura del dataframe, la normalizzazione della tabella consiste nel separare gli elementi presenti nella colonna lenders in modo che il dataframe sia formato da celle in cui è presente solo una istanza. Per farlo separo gli elementi che si trovano in corrispondenza della virgola, essendoci uno spazio vuoto dopo la virgola dovrò stare attento a considerarlo. Per ovviare a questo problema trasformo gli elementi ad ogni istanza di lenders in una lista.

In [5]:
loans_lenders['lenders'] = loans_lenders.apply(lambda row: row['lenders'].split(','), axis = 1)

Per separare gli elementi utilizzo la funzione *explode* implementata all'interno della libreria pandas che permette di sepoarare gli elementi di una lista lungo l'asse voluto.

In [6]:
loans_lenders_norm = loans_lenders.explode('lenders')

In [7]:
# Decommentare questa riga per vedere i risultati.
#loans_lenders_norm.head()

**2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.**

Come per il punto precedente procedo con una esplorazione preliminare del file in cui sono presenti gli attributi utili allo svolgimento del compito. Dopo averne fatto una prima visualizzazione degli attributi si nota che è utile passare gli attributi "disburse_time" e "planned_expiration_time" come date al momento dell'importazione.

In [8]:
loans = pd.read_csv(loans_path, parse_dates=["disburse_time", "planned_expiration_time"])

In [9]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [10]:
loans["duration"] = loans["planned_expiration_time"] - loans["disburse_time"]

Controllo per sicurezza quanti sono gli attributi nulli una volta fatta la differenza. In particolare è un numero abbastanza grosso, bisognerebbe quindi occuparsi di riempire i missing values.

In [11]:
loans.loc[loans['duration'].isnull() == True]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
64,251406,José Roan Group,Spanish,El grupo se llama “José Roan” y se ubica en l...,The group called “José Roan” is located in the...,825.0,825.0,funded,Poultry,Agriculture,...,7.0,22,2,1,NaN,"female, female, female","true, true, true",monthly,field_partner,NaT
65,252537,Marylou,English,"Marylou S. is from the village of Kuguita, Mam...",NaN,175.0,175.0,funded,Personal Products Sales,Retail,...,7.0,7,2,1,NaN,female,true,irregular,field_partner,NaT
66,253044,Flora,English,"Flora G. is from the village of Libona, Bukidn...",NaN,125.0,125.0,funded,Transportation,Transportation,...,7.0,5,2,1,NaN,female,true,irregular,field_partner,NaT
67,254012,Maria Magdalena,Spanish,"María de 55 años, tiene una tienda de víveres ...","María, 55, has a grocery store in the front pa...",1000.0,1000.0,funded,General Store,Retail,...,12.0,36,1,1,NaN,female,true,monthly,field_partner,NaT
69,251336,Rober,Spanish,"Rober, de 32 años, está casado con Janeth, de ...","Rober, 32, is married to Janeth, 31, and they ...",375.0,375.0,funded,Knitting,Arts,...,12.0,15,3,2,NaN,male,true,monthly,field_partner,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419523,288,Grace Ngima,English,<i>This business is a member of Naretoi Women ...,NaN,1200.0,1200.0,funded,Cattle,Agriculture,...,12.0,19,2,1,NaN,female,true,bullet,field_partner,NaT
1419524,1227124,Donna & Brighton,English,"In 1979, I graduated from the University of Wy...","In 1979, I graduated from the University of Wy...",800.0,10000.0,expired,Retail,Retail,...,36.0,9,1,1,NaN,female,true,bullet,direct,NaT
1419525,1089430,NaN,NaN,NaN,NaN,125.0,225.0,refunded,Grocery Store,Food,...,6.0,3,1,1,NaN,NaN,NaN,bullet,direct,NaT
1419538,1235452,Tara,English,Currently homeless.\r\nComplete alchemist.\r\n...,Currently homeless.\r\nComplete alchemist.\r\n...,600.0,3000.0,expired,Technology,Services,...,24.0,3,1,1,NaN,female,true,bullet,direct,NaT


**3) Find the lenders that have funded at least twice.**

Per fare questa cosa ho gli attributi già pronti. Per prima cosa bisogna effettuare un merge tra loans_lenders_norm e loans_ per sapere se tutte le istanze presenti in loans_lenders norm corrispondano allo stato "funded", una volta fatto questo mi occupo di tenere solo le righe in cui lo stato è funded e di selezionare solo quelle con un conteggio maggiore di o uguale a 2. Per non sovraccaricare troppo la RAM utilizzo per il merge solo gli attributi che mi serviranno per le computazioni.

In [12]:
loans_lenders_merged = pd.merge(loans_lenders_norm, loans[["loan_id", "status"]], on="loan_id", how="left")

In [13]:
loans_lenders_merged_funded = loans_lenders_merged[loans_lenders_merged["status"] == "funded"]

In [14]:
num_loan = loans_lenders_merged_funded.groupby('lenders').count()['loan_id']
num_loan.loc[num_loan >= 2]

lenders
 000               39
 00000             38
 0002              70
 0101craign0101    71
 0132575            4
                   ..
zyrah8525           4
zyrorl              3
zzaman              9
zzanita             2
zzmcfate           54
Name: loan_id, Length: 993782, dtype: int64

**4) For each country, compute how many loans have involved that country as borrowers.**

Durante l'esecuzione di questo punto nasce la problematica causata dal gran numero di NaN presenti nell'attributo duration notato durante l'esecuzione del punto 2. Per effettuare un calcolo corretto bisogna quindi mettere delle condizioni, in particolare creo un nuovo dataframe che contenga le righe di loans con le seguenti caratteristiche:

- Non hanno valori null in "duration".
- Per quanto riguarda i valori in "planned_expiration_time" che siano minori di "disburse_time" li scambio  (**DA FARE**). 

In [15]:
loans_not_null = loans[loans["duration"].isnull() == False]
loans_not_null = loans_not_null[loans_not_null["planned_expiration_time"] > loans_not_null["disburse_time"]]

Ora mi occuppo di raggruppare per country e di contare solo quelli corretti.

In [16]:
num_loans_country = loans_not_null.groupby('country_name').count()['loan_id']

In [17]:
#Decommentare questa riga per la visualizzazione dei risultati.
#num_loans_country

**5) For each country, compute the overall amount of money borrowed.** 

Per farlo è gia a disposizione il dataset sistemato, bisogna solo raggruppare e sommare per paese.

In [18]:
borr_country = loans_not_null.groupby('country_name')['loan_amount'].sum().reset_index().rename(columns={"loan_amount" : "borrowed_amount"})

In [19]:
#Decommentare questa riga per vedere i risultati
#borr_country

**6) Like the previous point, but expressed as a percentage of the overall amount lent.**

Allo stesso modo del punto precedente, basta stare attenti a fare il conto giusto.

In [20]:
overall_amount_lent = loans_not_null['loan_amount'].sum()
borr_country_perc = pd.DataFrame(loans_not_null.groupby('country_name')['loan_amount'].sum())
borr_country_perc['percentage'] = round(borr_country_perc['loan_amount'] / overall_amount_lent * 100, 2)
borr_country_perc = borr_country_perc.rename(columns={"loan_amount" : "loan_amount_perc"})

In [21]:
#Decommentare questa riga per vedere i risultati.
#borr_country_perc

Eseguo la somma per vedere che il risultato è unitario, ciò è vero a meno di approssimazioni che sono state svolte durante i conteggi:

In [23]:
round(borr_country_perc['percentage'].sum(),2)

99.98

**7) Like the three previous points, but split for each year (with respect to disburse time).**

Il primo passaggio è quello di riprendere il dataset creato in precedenza e settare l'attributo "disburse_time" come indice in modo da poterci applicare il metodo Grouper di pandas.

In [24]:
loans_not_null["disburse_time"] =  pd.to_datetime(loans_not_null["disburse_time"])
loans_not_null = loans_not_null.set_index("disburse_time")
loans_by_year_sum = loans_not_null.groupby(["country_name", pd.Grouper(freq="Y")])["loan_amount"].sum().to_frame()

Ora faccio le stesse cose fatte sopra ma calcolando la percentuale.

In [25]:
loans_by_year_perc = loans_by_year_sum
loans_by_year_perc["loan_amount"] = loans_by_year_perc["loan_amount"]/overall_amount_lent*100
loans_by_year_perc = loans_by_year_perc.rename(columns={"loan_amount" : "loan_amount_perc"})

Faccio come sopra in modo da poter contare questa volta.

In [26]:
loans_by_year_count = loans_not_null.groupby(["country_name", pd.Grouper(freq="Y")])["loan_id"].count().to_frame()
loans_by_year_count = loans_by_year_count.rename(columns={"loan_id" : "count"})

Eseguo il merge tra i tre dataframe selezionando solo le colonne che non sono duplicate in modo da avere il minor spreco di memoria possibile.

In [27]:
country_statistics_by_year = pd.concat([loans_by_year_count, loans_by_year_sum, loans_by_year_perc], axis=1)
country_statistics_by_year = country_statistics_by_year.loc[:, ~country_statistics_by_year.columns.duplicated()]

In [28]:
#Decommentare questa riga per stampare i risultati.
#display(country_statistics_by_year.head())

**8) For each lender, compute the overall amount of money lent.**

La prima cosa da fare è capire per ogni id il numero di lenders coinvolti, e ciò viene fatto contando il numero di persone presenti nel loan_id. L'assunzione fatta è che ogni lenders si sia occupato in egual misura di ogni loan.

In [29]:
lenders_num = loans_lenders_norm.groupby("loan_id").count().reset_index().rename(columns={"lenders" : "lenders_count"})

In [30]:
lenders_num_details = pd.merge(lenders_num, loans_not_null, on="loan_id")[["loan_id", "lenders_count", "loan_amount"]]

In [31]:
lenders_num_details["amount_per_person"] = lenders_num_details["loan_amount"] / lenders_num_details["lenders_count"]

In [32]:
loans_lenders_merged = pd.merge(loans_lenders_norm, lenders_num_details, on="loan_id", how="left")

In [33]:
lenders_overall_lent = loans_lenders_merged.groupby("lenders")["amount_per_person"].sum().to_frame().reset_index()

In [34]:
#Decommentare questa riga per vedere i risultati
#lenders_overall_lent.head()

**9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.**

La prima cosa da fare è sistemare il dataframe in modo da averlo senza valori NaN. Una volta diviso il dataframe in queste due parti cerco di calcolare la distribuzione nel dataframe in cui tutto è conosciuto e la applico al secondo dataframe.

In [35]:
lenders = pd.read_csv(lenders_path)

In [36]:
lenders_notnull = lenders.loc[lenders["country_code"].isnull() == False].reset_index()   
lenders_null = lenders.loc[lenders["country_code"].isnull() == True].reset_index()   

In [37]:
tot_notnull_users = len(lenders_notnull.index)
print(tot_notnull_users)

890539


Calcolo la distribuzione delle nazioni nel dataframe lenders_notnull

In [38]:
country_ripartition = lenders_notnull[["index", "country_code"]].groupby("country_code").count().reset_index().rename(columns = {"index":"n_users"})
country_ripartition["percentage"] = country_ripartition["n_users"]/tot_notnull_users*100

Ora riempio il dataframe lenders_null in modo che abbia la stessa distribuzione di lenders_notnull.
Per fare questo uso la funzione np.random.choice (inserendo come seed '1234')
In questo modo assegno la nazionalità in modo randomico e non sistematico (x. es. partendo dall'alto), ottenendo la distribuzione finale che voglio ottenere
ma per fare questo devo normalizzare le percentuali dividendole per la loro somma, altrimenti otterei l'errore (probabilities do not sum to 1

In [39]:
country_ripartition["percentage"] /= country_ripartition["percentage"].sum()

In [40]:
np.random.seed(1234)
lenders_null["country_code"] = np.random.choice(country_ripartition["country_code"], size=len(lenders_null.index), p = country_ripartition["percentage"])

In [41]:
lenders = pd.concat([lenders_notnull, lenders_null]).drop(columns="index")

Ora che il dataframe è sistemato con la distribuzione giusta, proseguo con la richiesta dell'esercizio.

Integro i dataframe "lenders", il quale ha i nomi dei lenders e la loro nazionalità, e il dataframe "lenders_overall_lent" che indica per ogni lender l'ammontare "lent".

A quel punto raggruppo per nazionalità e effettuo una somma, ottenendo l'ammontare "lent" per ogni nazione.

Inserisco queste info nel dataframe "tot_lent_country".


In [42]:
tot_lent_country = pd.merge(lenders_overall_lent, lenders[["permanent_name", "country_code"]], left_on="lenders", right_on="permanent_name")
tot_lent_country = tot_lent_country.drop(columns="permanent_name")
tot_lent_country = tot_lent_country.groupby("country_code")["amount_per_person"].sum()
tot_lent_country = tot_lent_country.to_frame().rename(columns={"amount_per_person" : "lent_amount"}).reset_index()

Nel dataframe "tot_lent_country" ho indicato il country_code, mentre nel dataframe "tot_borr_country" ho il nome della nazione, informazioni che mi servono entrambe. Eseguo quindi un merge per averle entrambe a portata di mano.

In [44]:
country_stats = pd.read_csv(country_stats_path)
loans_country = pd.merge(borr_country, country_stats[["country_name", "country_code"]], on="country_name")

Unisco ora le informazioni riguardo ai money_lent e ai money_borrowed in modo da poter eseguire la differenza tra questi due valori.

In [49]:
country_lent_borr = pd.merge(tot_lent_country, loans_country, on="country_code")
country_lent_borr = country_lent_borr[["country_name", "country_code", "lent_amount", "borrowed_amount"]]
country_lent_borr["difference"] = (country_lent_borr["lent_amount"] - country_lent_borr["borrowed_amount"])

In [52]:
#Decommentare questa riga per vedere i risultati
#country_lent_borr.head()

**10) Which country has the highest ratio between the difference computed at the previous point and the population?**

Per risolvere questo punto ho già tutte le informazioni che mi servono a portata di mano, le integro facendo un merge e poi svolgo una semplice query. Visto che ottengo differenze negative, sto attento a prendere il valore assoluto dell'indice costruito.

In [53]:
country_lent_borr  = pd.merge(country_lent_borr, country_stats[["country_code", "population"]], on="country_code")
country_lent_borr["ratio"] = abs(country_lent_borr["difference"]/country_lent_borr["population"])
max = country_lent_borr["ratio"].idxmax()
country_max = country_lent_borr.loc[max]

In [54]:
#Decommentare questa riga per vedere i risultati
country_max

country_name           Armenia
country_code                AM
lent_amount            1433.87
borrowed_amount    2.05793e+07
difference        -2.05778e+07
population             2930450
ratio                  7.02208
Name: 2, dtype: object